In [ ]:
import jax
import jax.numpy as np
import math
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

In [ ]:
# Creating a key for reproducibility
key = jax.random.PRNGKey(1)

<https://www.tensorflow.org/datasets/api_docs/python/tfds/load>

In [ ]:
# Import data
# 1150 files
import tensorflow_datasets as tfds

# tfds works in both Eager and Graph modes
# tf.enable_eager_execution()

# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
dataset = tfds.load(
    name="groove/full-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True)

In [ ]:
# Build your input pipeline
dataset = dataset.shuffle(1024).batch(32).prefetch(
    tf.data.experimental.AUTOTUNE)
for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]

In [ ]:
import tensorflow_datasets as tfds

# Construct a tf.data.Dataset
ds = tfds.load('mnist', split='train')

In [1]:
import numpy as np
# import tensorflow as tf
from pathlib import Path
from mido import MidiFile
import mido

seed = 42
# tf.random.set_seed(seed)
np.random.seed(seed)

class DataPoint:
    def __init__(self,
                 midi_data,
                 features,
                 audio_file_path,
                 midi_file_path,
                 purpose) -> None:
        self.midi_data = midi_data
        self.features = features
        self.audio_file_path = audio_file_path
        self.midi_file_path = midi_file_path
        self.purpose = purpose




feature_names = ["style", "bpm", "beat_type", "time_signature", "duration"]
feature_to_column_idx = {
    "session" : 0,
    "drummer" : 1,
    "id" : 2,
    "style" : 3,
    "bpm" : 4,
    "beat_type" : 5,
    "time_signature" : 6,
    "midi_filename" : 7,
    "audio_filename" : 8,
    "duration" : 9,
    "split" : 10
}

def play_audio(midi_data: MidiFile, duration: float=None) -> None:
        
    # Create output port for playing samples
    with mido.open_output(autoreset=True) as outport: 
        if isinstance(duration, (float, int)):
            for msg in midi_data.play():
                if duration <= 0: break
                outport.send(msg)
                duration -= msg.time
        else:
            for msg in midi_data.play():
                outport.send(msg)

    if not outport.closed:
        print("Closing port...")
        outport.close()

In [5]:



# Initiate data structures
X_train = [0] * 897
X_test = [0] * 129
X_validation = [0] * 124
# y_train = np.zeros(())
# y_test = np.zeros(())

# Define filename and its path
data_filename = "info.csv"
data_folder_path = Path().cwd() / "data"
data_path = data_folder_path / data_filename



# Open file with data information
with data_path.open() as file:
    columns = file.readline()
    train_idx = 0
    test_idx = 0
    validation_idx = 0
    for i, line in enumerate(file):
        # Strip line of new line and split into list
        line = line.strip().split(",")

        # Check
        # print(line)

        # Attach absolute path to midi & audio files
        midi_data_path = data_folder_path / line[7]
        audio_data_path = data_folder_path / line[8]

        # Create custom data point with all info
        midi_data = MidiFile(midi_data_path)
        features = {i: line[feature_to_column_idx[i]] for i in feature_names}
        data_point = DataPoint(
            midi_data=midi_data,
            audio_file_path=audio_data_path,
            midi_file_path=midi_data_path,
            features=features,
            purpose=line[feature_to_column_idx["split"]]
        )

        # Play the sample
        play_audio(midi_data, 3)

        if data_point.purpose == "train":
            X_train[train_idx] = data_point
            train_idx += 1
        elif data_point.purpose == "test":
            X_test[test_idx] = data_point
            test_idx += 1
        elif data_point.purpose == "validation":
            X_validation[validation_idx] = data_point
            validation_idx += 1

        # print(line.strip().split(","))
        print("Next sample\n")
        if i == 10: print("Breaking"); break

print(len(X_train))
print(len(X_test))
print(len(X_validation))

Next sample

Next sample

Next sample

Next sample

Next sample

Next sample

Next sample

Next sample

Next sample

Next sample

Next sample

Breaking
897
129
124


In [14]:
validation_idx

124

In [17]:
# print(X_train, X_test, X_validation, sep="\n")
# print(X_test[0])

for i in X_train[:1]:
    print(i.purpose, i.features, i.midi_file_path, i.audio_file_path, sep="\n")

train
{'style': 'funk', 'bpm': '80', 'beat_type': 'beat', 'time_signature': '4-4', 'duration': '86.704688'}
c:\Users\mrmar\Downloads\Neural Networks\Project\NN_Drummer_Beat_Project\data\drummer1\session1\1_funk_80_beat_4-4.mid
c:\Users\mrmar\Downloads\Neural Networks\Project\NN_Drummer_Beat_Project\data\drummer1\session1\1_funk_80_beat_4-4.wav


In [3]:
sample = X_train[0].midi_data
for i, track in enumerate(sample.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)

Track 0: 
MetaMessage('set_tempo', tempo=750000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='C', time=0)
program_change channel=9 program=0 time=0
program_change channel=9 program=0 time=0
note_on channel=9 note=38 velocity=7 time=1354
note_on channel=9 note=38 velocity=0 time=39
note_on channel=9 note=38 velocity=10 time=0
note_on channel=9 note=38 velocity=0 time=25
control_change channel=9 control=4 value=90 time=2
control_change channel=9 control=4 value=88 time=30
control_change channel=9 control=4 value=86 time=30
control_change channel=9 control=4 value=84 time=30
control_change channel=9 control=4 value=81 time=30
control_change channel=9 control=4 value=79 time=30
note_on channel=9 note=38 velocity=56 time=1
control_change channel=9 control=4 value=77 time=29
control_change channel=9 control=4 value=75 time=14
control_change channel=9 control=4 value=46 time=15
n

In [31]:
count = 0
print(len(X_validation))
for x in X_validation:
    count += 1 if x.midi_data.type == 0 else 0

print(count)

124
124


In [28]:
sample = X_train[0].midi_data
track = sample.tracks[0]

temp_msg = mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(120), time=0)

for i in sample.tracks:
    for j, msg in enumerate(i):
        if msg.type == "set_tempo":
            i[j] = mido.MetaMessage('set_tempo', tempo=mido.bpm2tempo(120), time=0)
        # msg.copy(tempo=100000)
        # print(dir(msg))
        # if j == 0: break
sample.save("test")

play_audio(mido.MidiFile("test"), 5)
# track.append(temp_msg)
# play_audio(sample, 5)